# Analyse des Aggregates Returns

## Importation

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

## Fonctions

In [15]:
def get_OLS(x,y):
    x_with_const = sm.add_constant(x)
    # Modélisation de la régression
    model = sm.OLS(y, x_with_const)
    results = model.fit()
    return results

In [16]:
def bootstrap_bias(data,params,nb) :
    betas = []
    stats = []
    rsquared = []
    for _ in range(nb):
        sample = data.sample(frac=1, replace=True)  # Resample with replacement
        res = get_OLS(sample['SSI'],sample.iloc[:,1])
        betas.append(res.params[1])
        stats.append(res.tvalues[1])
        rsquared.append(res.rsquared_adj)
    bias = np.mean(betas) - params[1]
    pvalue = np.mean(np.abs(stats))
    percentiles = [90, 95, 99]
    r2adj = [np.percentile(rsquared, p) for p in percentiles]
    return bias,pvalue,r2adj

## Données

In [17]:
SSI = pd.read_csv(r'C:\Users\collo\Documents\Etudes\Dauphine\API BLOOM\DAUPHINE_Python_Bloomberg\data\SSI.csv',index_col=0)
SSI.columns=['SSI']

In [18]:
SP500 = pd.read_excel(r"C:\Users\collo\Documents\Etudes\Dauphine\API BLOOM\DAUPHINE_Python_Bloomberg\data\sp500.xlsx",index_col=0)[['PX_LAST']]
SP500_lag = SP500
SP500=SP500.reset_index(drop=False).resample('M', on='Date').mean().pct_change()

## Comtemporaine

Full sample

In [19]:
# Données d'exemple
x = SSI['SSI'][:162]
y = SP500['PX_LAST'][6:168]
x.index = y.index
# Modélisation de la régression
results = get_OLS(x, y)
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                PX_LAST   R-squared:                       0.470
Model:                            OLS   Adj. R-squared:                  0.467
Method:                 Least Squares   F-statistic:                     142.0
Date:                Fri, 15 Mar 2024   Prob (F-statistic):           7.67e-24
Time:                        13:26:49   Log-Likelihood:                 360.38
No. Observations:                 162   AIC:                            -716.8
Df Residuals:                     160   BIC:                            -710.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0005      0.002      0.252      0.8

Post 2009

In [20]:
# Données d'exemple
x = SSI['SSI'][:162][30:]
y = SP500['PX_LAST'][6:168][30:]
x.index = y.index
results = get_OLS(x, y)
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                PX_LAST   R-squared:                       0.393
Model:                            OLS   Adj. R-squared:                  0.389
Method:                 Least Squares   F-statistic:                     84.27
Date:                Fri, 15 Mar 2024   Prob (F-statistic):           8.61e-16
Time:                        13:26:49   Log-Likelihood:                 306.15
No. Observations:                 132   AIC:                            -608.3
Df Residuals:                     130   BIC:                            -602.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0037      0.002      1.685      0.0

## Predictive

### Full sample

In [21]:
# Données d'exemple
x = SSI['SSI'][:162]
y = SP500['PX_LAST'][6:169].shift(-1).dropna()
x.index = y.index
# Modélisation de la régression
results = get_OLS(x, y)
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                PX_LAST   R-squared:                       0.280
Model:                            OLS   Adj. R-squared:                  0.276
Method:                 Least Squares   F-statistic:                     62.25
Date:                Fri, 15 Mar 2024   Prob (F-statistic):           4.49e-13
Time:                        13:26:49   Log-Likelihood:                 335.28
No. Observations:                 162   AIC:                            -666.6
Df Residuals:                     160   BIC:                            -660.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0020      0.002      0.820      0.4

### post - 2009

In [22]:
# Données d'exemple
x = SSI['SSI'][:162][30:]
y = SP500['PX_LAST'][6:169][30:].shift(-1).dropna()
x.index = y.index
# Modélisation de la régression
results = get_OLS(x, y)
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                PX_LAST   R-squared:                       0.218
Model:                            OLS   Adj. R-squared:                  0.212
Method:                 Least Squares   F-statistic:                     36.34
Date:                Fri, 15 Mar 2024   Prob (F-statistic):           1.60e-08
Time:                        13:26:49   Log-Likelihood:                 289.50
No. Observations:                 132   AIC:                            -575.0
Df Residuals:                     130   BIC:                            -569.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0057      0.003      2.290      0.0

In [23]:
data = pd.DataFrame({
    'SSI': x,
    'PX_LAST': y})
bootstrap_bias(data,results.params,1000)

(0.0001077360176933162,
 6.105053473995205,
 [0.30838154746285734, 0.3362736257265767, 0.3821108800232682])